### XXXXXX

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [2]:
from utils.returns_data_class import ReturnsData
PERIOD = 1
data = ReturnsData(
    daily_returns_path="../Data/returns_df_611.csv",
    extras_path="../Data/historical_stocks.csv",
)
data.change_returns_period(PERIOD)
X = data.returns_df.values.T

No change made because period entered is 1


In [3]:
num_TS = X.shape[0]
num_pos_samples = 10
period = 10
stride = 3
num_neg_samples = 30

print(f"Context Size: {num_pos_samples}, Period: {period}, Stride: {stride}")
print(f"Number of Negative Samples: {num_neg_samples}")

from utils.context import get_tgt_context_euclidean_multiprocess
positive_tgt_context_sets = get_tgt_context_euclidean_multiprocess(ts_array=X, m=period, k=num_pos_samples, stride=stride, z_normalize=False, st_dev_pruning=False, verbose=True)
negative_tgt_context_sets = get_tgt_context_euclidean_multiprocess(ts_array=X, m=period, k=num_neg_samples, stride=stride, z_normalize=False, st_dev_pruning=False, top_k=False, verbose=True)
print(f"Number (anchor, positive, negative) samples: {len(positive_tgt_context_sets)}")

Context Size: 10, Period: 10, Stride: 3
Number of Negative Samples: 30


100%|██████████| 87/87 [00:08<00:00, 10.84it/s]


nearly returning
Number (anchor, positive, negative) samples: 953771


In [4]:
import pandas as pd
import numpy as np
ticker = "JPM"
i = data.ticker2idx[ticker]
temp = np.array([xi[1] for xi in positive_tgt_context_sets if xi[0] == i]).flatten()
print(
    f"The most commonly co-occurring stocks with {ticker} are: {[data.idx2ticker[xi] for xi in pd.Series(temp).value_counts().index][:5]}"
)
temp = pd.Series(temp).value_counts()
temp.index = temp.index.map(data.idx2ticker)
temp = temp/temp.sum()
temp.iloc[:5]

The most commonly co-occurring stocks with JPM are: ['C', 'BAC', 'WFC', 'USB', 'PNC']


C      0.030750
BAC    0.026393
WFC    0.024920
USB    0.023959
PNC    0.020884
Name: count, dtype: float64

Sample positive and negative pairs based on Chi-Squared p-value. If cooccurrence is more than random with high confidence then sample more.



In [5]:
from tqdm import tqdm

def get_cooccurrence_counts(tgt_context_sets, data:ReturnsData):
    distributions = {}
    for ticker in tqdm(data.tickers):
        i = data.ticker2idx[ticker]
        all_samples = np.array([xi[1] for xi in tgt_context_sets if xi[0] == i]).flatten()
        sample_count = pd.Series(all_samples).value_counts()
        sample_count.index = sample_count.index.map(data.idx2ticker)
        sample_count = sample_count.to_dict()
        # Add zero cooccurrences
        zero_cooccurrences = set(data.tickers)-set(sample_count.keys())-set([ticker])
        sample_count.update(dict.fromkeys(list(zero_cooccurrences), 0))

        distributions[ticker] = sample_count
    return distributions
positive_sample_distributions = get_cooccurrence_counts(positive_tgt_context_sets, data)

  0%|          | 0/611 [00:00<?, ?it/s]

100%|██████████| 611/611 [00:35<00:00, 17.20it/s]


In [8]:
import pandas as pd
from scipy.stats import norm
from typing import Literal

def test_ticker_cooccurrence_significance(
    t1: str,
    t2: str,
    distributions: dict,
    test_direction: Literal["positive_samples", "negative_samples"] = "positive_samples",
    alpha: float | None = None,
    verbose: bool = False,
):
    # Convert to DataFrame
    df = pd.DataFrame(distributions).fillna(0)

    # Calculate total counts
    total_counts = df.sum().sum()

    # Observed count for word_b with word_2
    observed_count = df.loc[t2, t1]

    # Expected count under equal frequency assumption
    # (1/num_TS)*int((X.shape[1]-period)/stride) * num_pos_samples
    expected_count = df[t1].sum() / len(df)

    # Perform a Z-test for proportions
    test_statistic = (observed_count - expected_count) / np.sqrt(
        expected_count * (1 - expected_count / total_counts)
    )
    # p_value = norm.sf(abs(z_score))  # two-tailed test
    if test_direction=="positive_samples":
        p_value = norm.sf(test_statistic)  # one-tailed test
    elif test_direction=="negative_samples":
        p_value = norm.cdf(test_statistic)

    if verbose:
        print(f"Observed Count: {observed_count}")
        print(f"Expected Count: {expected_count}")
        print(f"Test Statistic: {test_statistic}")
        print(f"P-value: {p_value}")
    if alpha is None:
        return p_value
    else:
        return p_value < alpha

test_ticker_cooccurrence_significance("JPM", "C", positive_sample_distributions, verbose=True, test_direction="positive_samples")

Observed Count: 480.0
Expected Count: 25.548281505728315
Test Statistic: 89.90989392471835
P-value: 0.0


0.0

In [19]:
samples = []

for ticker in tqdm(data.tickers):
    for t, c in positive_sample_distributions[ticker].items():
        p_value_positive = test_ticker_cooccurrence_significance(
            ticker,
            t,
            positive_sample_distributions,
            test_direction="positive_samples",
        )

        samples.append((ticker,t,c,p_value_positive))


100%|██████████| 611/611 [5:29:09<00:00, 32.32s/it]  


In [20]:
import pickle
with open('samples.pkl', 'wb') as f:
    pickle.dump(samples, f)

Load samples

In [52]:
samples_df = pd.DataFrame(samples, columns=["query_ticker", "sample_ticker", "count", "p_value"])
samples_df["query_ticker_idx"] = samples_df["query_ticker"].map(data.ticker2idx)
samples_df["sample_ticker_idx"] = samples_df["sample_ticker"].map(data.ticker2idx)

In [109]:
def get_distribution(ticker, samples_df, sample_type:Literal["positive_samples", "negative_samples"]="positive_samples"):
    filtered_df = samples_df[samples_df["query_ticker"]==ticker].copy()
    if sample_type=="positive_samples":
        filtered_df = filtered_df[filtered_df["p_value"]<0.025].copy()
    elif sample_type=="negative_samples":
        filtered_df = filtered_df[filtered_df["p_value"]>0.7].copy()
    else:
        raise ValueError("Invalid sample_type")
    filtered_df["pos_prob"] = filtered_df["count"]/filtered_df["count"].sum()
    filtered_df["pos_prob_sq"] = filtered_df["pos_prob"]**2
    filtered_df["pos_prob_sq"] = filtered_df["pos_prob_sq"]/filtered_df["pos_prob_sq"].sum()
    # filtered_df["pos_prob_sq"] = filtered_df["pos_prob_sq"].round(3)
    filtered_distribution = filtered_df[["query_ticker", "sample_ticker", "pos_prob_sq"]].values
    return filtered_distribution

In [110]:
def sample_tgt_context_sets(filtered_distribution, n_samples, sample_size):
    # Unique first elements
    unique_first_elements = np.unique(filtered_distribution[:, 0])
    if len(unique_first_elements)>1:
        raise ValueError("Should only be a single distribution")
    first_elem = unique_first_elements[0]

    sampled_pairs = []

    # Filter distribution for the current first element
    current_distribution = filtered_distribution[filtered_distribution[:, 0] == first_elem]

    # Check if there are enough elements to sample
    if len(current_distribution) < sample_size:
        raise ValueError(f"Not enough elements to sample {sample_size} times for '{first_elem}'")

    # Extract second elements and probabilities
    second_elements = current_distribution[:, 1]
    probabilities = current_distribution[:, 2].astype(float)

    # Sample multiple times
    # samples_for_first_elem = []
    for _ in range(n_samples):
        # Sample without replacement
        sampled_indices = np.random.choice(len(probabilities), size=sample_size, replace=False, p=probabilities)
        sampled_second_elements = second_elements[sampled_indices]
        # samples_for_first_elem.append(list(sampled_second_elements))

        # Append to result
        sampled_pairs.append((first_elem, list(sampled_second_elements)))

    return sampled_pairs


In [112]:

num_neg_samples = 20
num_pos_samples = 5
num_sets = 1000
dist_shapes = []
positive_tgt_context_sets = []
negative_tgt_context_sets = []
for ticker in tqdm(data.tickers):
    temp_ticker_distribution_pos = get_distribution(ticker, samples_df, sample_type="positive_samples")
    temp_ticker_distribution_neg = get_distribution(ticker, samples_df, sample_type="negative_samples")
    dist_shapes.append((temp_ticker_distribution_pos.shape[0], temp_ticker_distribution_neg.shape[0]))
    temp_ticker_positive_sets = sample_tgt_context_sets(temp_ticker_distribution_pos, n_samples=num_sets, sample_size=num_pos_samples)
    temp_ticker_negative_sets = sample_tgt_context_sets(temp_ticker_distribution_neg, n_samples=num_sets, sample_size=num_neg_samples)
    positive_tgt_context_sets += temp_ticker_positive_sets
    negative_tgt_context_sets += temp_ticker_negative_sets

100%|██████████| 611/611 [00:43<00:00, 13.95it/s]


In [113]:
import plotly.express as px
px.histogram(np.array(dist_shapes)[:,1])

In [91]:
import plotly.express as px
fig = px.scatter(temp)
fig.update_layout(template="plotly_dark")

In [66]:
import plotly.express as px
fig = px.scatter(temp)
fig.update_layout(template="plotly_dark")

In [114]:
index_samples = []
for pos, neg in zip(positive_tgt_context_sets,negative_tgt_context_sets):
    # index_samples.append((pos[0], pos[1], neg[1]))
    index_samples.append((data.ticker2idx[pos[0]], [data.ticker2idx[xi] for xi in pos[1]], [data.ticker2idx[xi] for xi in neg[1]]))

### Experimental Contrastive Loss

In [115]:
import torch
from torch.utils.data import Dataset, DataLoader


class MultiPosNegDataset(Dataset):
    def __init__(self, index_samples):
        self.index_samples = index_samples

    def __len__(self):
        return len(self.index_samples)

    def __getitem__(self, idx):
        anchor_idx, positive_indices, negative_indices = self.index_samples[idx]
        # Convert indices list to tensor

        positive_indices_tensor = torch.tensor(positive_indices)
        negative_indices_tensor = torch.tensor(negative_indices)
        return anchor_idx, positive_indices_tensor, negative_indices_tensor


def normalize_embeddings(embeddings):
    with torch.no_grad():  # Ensure no gradients are computed in this block
        norms = embeddings.weight.norm(dim=1, keepdim=True)
        embeddings.weight.data = embeddings.weight.data / norms
    return embeddings


# def experimental_loss(
#     anchor_embeddings: torch.Tensor,
#     positive_embeddings: torch.Tensor,
#     negative_embeddings: torch.Tensor,
# ) -> torch.Tensor:
#     """

#     Args:
#         anchor_embeddings (torch.Tensor): shape (batch_size, embedding_dim)
#         positive_embeddings (torch.Tensor): (batch_size, num_pos_samples, embedding_dim)
#         negative_embeddings (torch.Tensor): (batch_size, num_neg_samples, embedding_dim)

#     Returns:
#         torch.Tensor: _description_
#     """
#     positive_scores = torch.einsum(
#             "bpd,bd->bp", [positive_embeddings, anchor_embeddings]
#         )
#     negative_scores = torch.einsum(
#             "bnd,bd->bn", [negative_embeddings, anchor_embeddings]
#         )
#     # positive_loss = - torch.sum(torch.nn.functional.logsigmoid(positive_scores), dim=1)
#     # negative_loss = - torch.sum(torch.log(1-torch.sigmoid(negative_scores)), dim=1)
#     criterion = torch.nn.BCEWithLogitsLoss()
#     positive_loss = criterion(positive_scores, torch.ones_like(positive_scores))
#     negative_loss = criterion(negative_scores, torch.zeros_like(negative_scores))
#     loss = torch.sum(positive_loss + negative_loss)

#     return loss

def experimental_loss(
    anchor_embeddings: torch.Tensor,
    positive_embeddings: torch.Tensor,
    negative_embeddings: torch.Tensor,
) -> torch.Tensor:
    """

    Args:
        anchor_embeddings (torch.Tensor): shape (batch_size, embedding_dim)
        positive_embeddings (torch.Tensor): (batch_size, num_pos_samples, embedding_dim)
        negative_embeddings (torch.Tensor): (batch_size, num_neg_samples, embedding_dim)

    Returns:
        torch.Tensor: _description_
    """
    positive_scores = torch.einsum(
            "bd,bd->b", [torch.mean(positive_embeddings, dim=1), anchor_embeddings]
        )
    negative_scores = torch.einsum(
            "bd,bd->b", [torch.mean(negative_embeddings, dim=1), anchor_embeddings]
        )
    # positive_loss = - torch.sum(torch.nn.functional.logsigmoid(positive_scores))
    # negative_loss = - torch.sum(torch.log(1-torch.sigmoid(negative_scores)+0.00001))
    criterion = torch.nn.BCEWithLogitsLoss()
    positive_loss = criterion(positive_scores, torch.ones_like(positive_scores))
    negative_loss = criterion(negative_scores, torch.zeros_like(negative_scores))

    loss = positive_loss + negative_loss

    return loss

# def experimental_loss(
#     anchor_embeddings: torch.Tensor,
#     positive_embeddings: torch.Tensor,
#     negative_embeddings: torch.Tensor,
# ) -> torch.Tensor:
#     """

#     Args:
#         anchor_embeddings (torch.Tensor): shape (batch_size, embedding_dim)
#         positive_embeddings (torch.Tensor): (batch_size, num_pos_samples, embedding_dim)
#         negative_embeddings (torch.Tensor): (batch_size, num_neg_samples, embedding_dim)

#     Returns:
#         torch.Tensor: _description_
#     """
#     ### POSITIVE LOSS - Sigmoid over each positive
#     positive_scores = torch.einsum(
#             "bpd,bd->bp", [positive_embeddings, anchor_embeddings]
#         )
#     criterion = torch.nn.BCEWithLogitsLoss()
#     positive_loss = criterion(positive_scores, torch.ones_like(positive_scores))

#     ### NEGATIVE LOSS - Softmax with aggregate positive representation
#     criterion = torch.nn.NLLLoss()
#     aggregate_positive_embeddings = torch.mean(positive_embeddings, dim=1) # (batch_size, embedding_dim)
#     aggregate_positive_scores = torch.einsum(
#             "bd,bd->b", [aggregate_positive_embeddings, anchor_embeddings]
#         )
#     # aggregate_positive_scores = torch.mean(positive_scores, dim=1)
#     negative_scores = torch.einsum(
#             "bnd,bd->bn", [negative_embeddings, anchor_embeddings]
#         )
#     #-- Make first column the aggregate positive scores
#     #- concatenated.shape = (batch_size, 1+num_neg_samples)
#     concatenated = torch.concat((aggregate_positive_scores.unsqueeze(1), negative_scores), dim=1)
#     #-- Create target for NLLLoss
#     target = torch.zeros(concatenated.shape[0], dtype=torch.long)

#     negative_loss = criterion(torch.nn.functional.log_softmax(concatenated, dim=1), target)
#     loss = positive_loss + negative_loss

#     return loss

### Training

In [116]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Parameters
num_items = num_TS  # Number of items in your dataset
embedding_size = 16  # Size of each embedding
learning_rate = 0.001
epochs = 10
batch_size = 2048

# Initialize the embedding matrix
embeddings = nn.Embedding(num_embeddings=num_items, embedding_dim=embedding_size)

# Optimizer
optimizer = optim.Adam(embeddings.parameters(), lr=learning_rate)

# Prepare your index_samples as a tensor
# index_samples = torch.tensor([(35, 12, 98), (47, 12, 4), ...])  # Your index tuples as a tensor
# Create the dataset and data loader
npair_dataset = MultiPosNegDataset(index_samples)
data_loader = DataLoader(npair_dataset, batch_size=batch_size, shuffle=True)

losses = []
REGULARIZATION_WEIGHT = 1
# Training loop with batching
for epoch in range(epochs):
    total_loss = 0
    # normalise the embeddings to prevent degenerate solution
    embeddings = normalize_embeddings(embeddings)

    for i, (anchor_idx, positive_idx, negative_indices) in enumerate(data_loader):

        # Get the embeddings for anchor, positive, and negative
        anchor_embeddings = embeddings(anchor_idx) # shape: (batch_size, embedding_dim)
        positive_embeddings = embeddings(positive_idx) # shape: (batch_size, num_pos_samples embedding_dim)
        negative_embeddings = embeddings(negative_indices) # shape: (batch_size, num_neg_samples, embedding_dim)

        # Compute the loss
        loss = experimental_loss(anchor_embeddings, positive_embeddings, negative_embeddings)
        regularization_loss = REGULARIZATION_WEIGHT * torch.square(embeddings.weight.norm(dim=1) - 1).sum()
        loss = loss + regularization_loss
        total_loss += loss.item()
        losses.append((loss.item(), regularization_loss.item()))

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch % 1 == 0) | (epoch==epochs-1) | (epoch==0):
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(data_loader)}')

embeddings = normalize_embeddings(embeddings)

Epoch [1/10], Loss: 1.3234279422855697
Epoch [2/10], Loss: 1.1308365246922676
Epoch [3/10], Loss: 1.0473937370306672
Epoch [4/10], Loss: 1.027993889955374
Epoch [5/10], Loss: 1.0229337490521944
Epoch [6/10], Loss: 1.021214877881334


KeyboardInterrupt: 

In [14]:
import plotly.express as px
losses=np.array(losses)
px.scatter(losses[:,0])

In [130]:
px.scatter(losses[:,1])

### Evaluation

In [118]:
from utils.sector_classification import get_sector_score

get_sector_score(embeddings.weight.detach().numpy(), sectors=data.sectors, top_k_accuracy=True)

/opt/homebrew/Caskroom/miniforge/base/envs/phd/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Precision Score: 0.63
Recall Score: 0.57
F1 Score: 0.57
Accuracy Score: 0.57
Accuracy Score Top-3: 0.82


In [119]:
from utils.visualisation_functions import pca_plot_from_embeddings

pca_plot_from_embeddings(
    embedding_matrix=embeddings.weight.detach().numpy(),
    sectors=data.sectors,
    tickers=data.tickers,
    industries=data.industries,
    names=data.names,
    dimensions=2,
    reduced=True,
    method="PCA",
    return_df=False,
    rand_state=None,
)

In [121]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
embeddings = embeddings.weight.detach()
train_size = int(len(data.sectors)*0.7)
X_train = embeddings[:train_size, :]
X_test = embeddings[train_size:, :]
y_train = data.sectors[:train_size]
y_test = data.sectors[train_size:]

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from utils.ucr_helpers import evaluate_resampling_UCR
report, _, _ = evaluate_resampling_UCR(
    X_train,
    X_test,
    y_train,
    y_test,
    # classifier=SVC(kernel="rbf"),
    # classifier=LogisticRegression(),
    classifier=KNeighborsClassifier(n_neighbors=1),
    # classifier=MLPClassifier(),
    n_resamples=20,
    verbose=True,
    scale=False,
    over_sampling=True
)
report

100%|██████████| 19/19 [00:00<00:00, 42.47it/s]


{'precision': 0.6288955675728009,
 'recall': 0.6059782608695651,
 'f1-score': 0.6118027998776332,
 'accuracy': 0.6059782608695652}